In [1]:
import pandas as pd


In [14]:
df = pd.read_excel("DATA\clean_Books.xlsx")


In [15]:
df.shape

(271379, 6)

In [16]:
df.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,"""Classical Mythology""","""Mark P. O. Morford""","""2002""","""Oxford University Press""","""http://images.amazon.com/images/P/0195153448...."
1,0002005018,"""Clara Callan""","""Richard Bruce Wright""","""2001""","""HarperFlamingo Canada""","""http://images.amazon.com/images/P/0002005018...."


In [5]:
# Check the number of split values
num_values = df['ISBN;"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"'].str.split(';').apply(len).max()

# Create a list of column names based on the number of split values
columns = ['col{}'.format(i) for i in range(num_values)]

# Split the single column into multiple columns using the semicolon delimiter
df[columns] = df['ISBN;"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"'].str.split(';', expand=True)

# Drop the original column
df.drop(columns=['ISBN;"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"'], inplace=True)


In [100]:
users = pd.read_csv("DATA\clean_users.xlsx", sep=';', encoding="latin-1")


In [101]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [98]:
users.to_excel("clean_users.xlsx", index=False)

In [97]:
users.shape

(278858, 3)

In [2]:
ratings = pd.read_csv("DATA\BX-Book-Ratings.csv", sep=';', encoding="latin-1")

In [3]:
ratings.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [4]:
ratings.shape

(1149780, 3)

In [5]:
x=ratings['User-ID'].value_counts() > 200
x[x].shape

(899,)

In [6]:
y= x[x].index
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='User-ID', length=899)

In [11]:
ratings = ratings[ratings['User-ID'].isin(y)]

In [12]:
ratings.head()

,User-ID,ISBN,Book-Rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [18]:
ratings_with_books = ratings.merge(df, on='ISBN')

In [20]:
ratings_with_books.shape

(487685, 8)

In [23]:
ratings_with_books.rename(columns={'BOOK-Rating': 'rating'}, inplace=True)
ratings_with_books.rename(columns={'USER-ID': 'user_id'}, inplace=True)

ratings_with_books.rename(columns={'Book-Title': 'Title'}, inplace=True)
ratings_with_books.rename(columns={'Book-Author': 'Author'}, inplace=True)
ratings_with_books.rename(columns={'Year-Of-Publication': 'year'}, inplace=True)

ratings_with_books.rename(columns={'Image-URL-L': 'img'}, inplace=True)



In [29]:
number_rating = ratings_with_books.groupby('Title')['Book-Rating'].count().reset_index()
number_rating.head()

,Title,Book-Rating
0,""" A Light in the Storm: The Civil War Diary of...",2
1,""" Always Have Popsicles""",1
2,""" Apple Magic (The Collector's series)""",1
3,""" Beyond IBM: Leadership Marketing and Finance...",1
4,""" Clifford Visita El Hospital (Clifford El Gra...",1


In [32]:
number_rating.rename(columns={'Book-Rating':'num_of_rating'},inplace=True)
number_rating.head()

,Title,num_of_rating
0,""" A Light in the Storm: The Civil War Diary of...",2
1,""" Always Have Popsicles""",1
2,""" Apple Magic (The Collector's series)""",1
3,""" Beyond IBM: Leadership Marketing and Finance...",1
4,""" Clifford Visita El Hospital (Clifford El Gra...",1


In [34]:
final_rating = ratings_with_books.merge(number_rating, on='Title')
final_rating.head()


,User-ID,ISBN,Book-Rating,Title,Author,year,Publisher,img,num_of_rating
0,277427,002542730X,10,"""Politically Correct Bedtime Stories: Modern T...","""James Finn Garner""","""1994""","""John Wiley &amp","""http://images.amazon.com/images/P/002542730X....",82
1,3363,002542730X,0,"""Politically Correct Bedtime Stories: Modern T...","""James Finn Garner""","""1994""","""John Wiley &amp","""http://images.amazon.com/images/P/002542730X....",82
2,11676,002542730X,6,"""Politically Correct Bedtime Stories: Modern T...","""James Finn Garner""","""1994""","""John Wiley &amp","""http://images.amazon.com/images/P/002542730X....",82
3,12538,002542730X,10,"""Politically Correct Bedtime Stories: Modern T...","""James Finn Garner""","""1994""","""John Wiley &amp","""http://images.amazon.com/images/P/002542730X....",82
4,13552,002542730X,0,"""Politically Correct Bedtime Stories: Modern T...","""James Finn Garner""","""1994""","""John Wiley &amp","""http://images.amazon.com/images/P/002542730X....",82


In [35]:
final_rating.shape


(487685, 9)

In [36]:
final_rating = final_rating[final_rating['num_of_rating'] >= 50]
final_rating.shape


(63001, 9)

In [40]:
# lets drop the duplicates
final_rating.drop_duplicates(['User-ID','Title'],inplace=True)
final_rating.shape

(60793, 9)

In [41]:
final_rating.to_excel("clean_rating.xlsx", index=False)